In [ ]:
!wget -c "https://onedrive.live.com/download?cid=A5E3C415BF70F5D8&resid=A5E3C415BF70F5D8%217785&authkey=AP5Xz9M015K8zbc" -O "onnx_partial.tar.xz"

--2022-05-22 10:57:10--  https://onedrive.live.com/download?cid=A5E3C415BF70F5D8&resid=A5E3C415BF70F5D8%217785&authkey=AP5Xz9M015K8zbc
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://piqdcg.db.files.1drv.com/y4m4MdFww-YSIm8Ya9RgRaxpE6msx4t-H8VP9nX6Om05yBxqtUZvqn9_6kuF5TJ6THRe-ZoanOMex5F3vifEgodR-6JJzRXRaPc8Q-7T9iet18TDsZhQ2qxnI2sR9k1u7qty_Cb1xE0vEFx-uaNSZK7a_8Hx0h25xSnyjMzXixS-hwNEHUoae47DatbW2XpiSu6dJCYOKz6WJtb1VSNo8-IMA/onnx_partial.tar.xz?download&psid=1 [following]
--2022-05-22 10:57:11--  https://piqdcg.db.files.1drv.com/y4m4MdFww-YSIm8Ya9RgRaxpE6msx4t-H8VP9nX6Om05yBxqtUZvqn9_6kuF5TJ6THRe-ZoanOMex5F3vifEgodR-6JJzRXRaPc8Q-7T9iet18TDsZhQ2qxnI2sR9k1u7qty_Cb1xE0vEFx-uaNSZK7a_8Hx0h25xSnyjMzXixS-hwNEHUoae47DatbW2XpiSu6dJCYOKz6WJtb1VSNo8-IMA/onnx_partial.tar.xz?download&psid=1
Resolving piqdcg.db.files.1drv.com (piqdcg.db.

In [ ]:
!mkdir /content/models/
!tar -xf onnx_partial.tar.xz -C /content/models/

In [ ]:
!git clone https://github.com/davidhoof/master-thesis-automatic-selection.git

Cloning into 'master-thesis-automatic-selection'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 196 (delta 9), reused 48 (delta 9), pack-reused 147
Receiving objects: 100% (196/196), 48.29 MiB | 29.65 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [ ]:
!pip install onnx

     |████████████████████████████████| 12.8 MB 5.3 MB/s 


In [ ]:
import onnx
import sys
import os
from tqdm import tqdm

In [ ]:
import glob
paths = glob.glob('/content/models/onnx_zoo/**/*.onnx', recursive=True)

In [ ]:
len(paths)

487

In [ ]:
def convertFloatToDecimal(f=0.0, precision=2):
    '''
    Convert a float to string of decimal.
    precision: by default 2.
    If no arg provided, return "0.00".
    '''
    return ("%." + str(precision) + "f") % f

def formatFileSize(size, sizeIn, sizeOut, precision=0):
    '''
    Convert file size to a string representing its value in B, KB, MB and GB.
    The convention is based on sizeIn as original unit and sizeOut
    as final unit. 
    '''
    assert sizeIn.upper() in {"B", "KB", "MB", "GB"}, "sizeIn type error"
    assert sizeOut.upper() in {"B", "KB", "MB", "GB"}, "sizeOut type error"
    if sizeIn == "B":
        if sizeOut == "B":
          return size
        if sizeOut == "KB":
            return convertFloatToDecimal((size/1024.0), precision)
        elif sizeOut == "MB":
            return convertFloatToDecimal((size/1024.0**2), precision)
        elif sizeOut == "GB":
            return convertFloatToDecimal((size/1024.0**3), precision)
    elif sizeIn == "KB":
        if sizeOut == "B":
            return convertFloatToDecimal((size*1024.0), precision)
        elif sizeOut == "MB":
            return convertFloatToDecimal((size/1024.0), precision)
        elif sizeOut == "GB":
            return convertFloatToDecimal((size/1024.0**2), precision)
    elif sizeIn == "MB":
        if sizeOut == "B":
            return convertFloatToDecimal((size*1024.0**2), precision)
        elif sizeOut == "KB":
            return convertFloatToDecimal((size*1024.0), precision)
        elif sizeOut == "GB":
            return convertFloatToDecimal((size/1024.0), precision)
    elif sizeIn == "GB":
        if sizeOut == "B":
            return convertFloatToDecimal((size*1024.0**3), precision)
        elif sizeOut == "KB":
            return convertFloatToDecimal((size*1024.0**2), precision)
        elif sizeOut == "MB":
            return convertFloatToDecimal((size*1024.0), precision)

In [ ]:
def get_size_range(size):    
  if size/1024>1 and size/1024<=1024:
    return "KB"
  if size/1024.0**2>1 and size/1024.0**2<=1024:
    return "MB"
  if size/1024.0**3>1 and size/1024.0**3<=1024:
    return "GB"
  return "B"

In [ ]:
get_size_range(1024)

'B'

In [ ]:
formatFileSize(1024,'B',get_size_range(1024),2)

1024

In [ ]:
file_dict={os.path.basename(path)[:-5] : str(formatFileSize(os.path.getsize(path),'B',get_size_range(os.path.getsize(path)),2))+get_size_range(os.path.getsize(path)) for path in tqdm(paths)}

100%|██████████| 487/487 [00:00<00:00, 44462.91it/s]


In [ ]:
import pandas as pd
df=pd.DataFrame.from_dict(file_dict, orient='index')
df.reset_index(inplace=True)
df.rename(columns={"index":"Name",0:"Filesize"},inplace=True)

In [ ]:
df

,Name,Filesize
0,cyclegan_style_vangogh_g_11,43.47MB
1,mealv2_mobilenetv3_small_100_imagenet_11,9.71MB
2,mealv1_resnest50_imagenet_11,97.40MB
3,pytorchencoding_deeplab_resnest101_pcontext_11,246.79MB
4,cyclegan_cityscapes_photo2label_g_11,43.47MB
...,...,...
482,squeezenet1_0_imagenet_11,4.77MB
483,vgg13_imagenet_11,507.54MB
484,vgg16_bn_imagenet_11,527.80MB
485,wide_resnet101_2_imagenet_11,483.79MB


In [ ]:
df.to_csv("file_size_onnx_zoo.csv")